## Question 1a

$1+\frac{1}{1+\frac{1}{1+\frac{1}{1+\frac{1}{1+\frac{1}{1+⋱}}}}}$

## Question 1b
1. [book](book.com)
2. book
3. book
4. book
5. book

## Question 1c
<img href = ""></img>

## Question 1d

## Question 2a

In [8]:
function get_roots(a,b,c)
    d::Float64 = b^2 - 4*a*c
    if d < 0
        return []
    elseif d == 0
        return [-b/(2*a)]
    else
        return [(-b+sqrt(d))/(2*a), (-b-sqrt(d))/(2*a)]
    end
end

get_roots(-1,5,-6)

2-element Vector{Float64}:
 2.0
 3.0